In [1]:
# from audio_set_loading import *
import sys, time, librosa, os, argparse, pickle
from tqdm import tqdm
sys.path.append('..')
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
import audio_utils
from utils import get_laughter_infolist, download_laughter_audio_file
from constants import audioset_data_path

In [3]:
audioset_train_path='/mnt/e/data/audioset/unbalanced_train'
audioset_test_path='/mnt/e/data/audioset/eval'
audioset_train_df_path = 'data/audioset/unbalanced_train_segments.csv'
audioset_train_df = pd.read_csv(audioset_train_df_path, quotechar='"', skipinitialspace=True)
audioset_test_df_path = 'data/audioset/eval_segments.csv'
audioset_test_df = pd.read_csv(audioset_test_df_path, quotechar='"', skipinitialspace=True)
out_dir = os.path.join(audioset_data_path, 'joint')

In [10]:
train_examples = get_laughter_infolist(audioset_train_df_path, 'both')
test_examples = get_laughter_infolist(audioset_test_df_path, 'both')
all_examples = train_examples + test_examples
all_example_ids = [ex['yt_id'] for ex in all_examples]

In [5]:

Parallel(n_jobs=8)(delayed(download_laughter_audio_file)(ex, out_dir) for ex in tqdm(all_examples))

100%|██████████| 19354/19354 [1:47:26<00:00,  3.00it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [4]:
all_audioset_files = librosa.util.find_files(out_dir, ext=['mp3'])

In [5]:
audios = audio_utils.parallel_load_audio_batch(all_audioset_files, n_processes=8, sr=8000)

100%|██████████| 15947/15947 [15:50<00:00, 16.77it/s]


In [7]:
h = {}
for i in range(len(all_audioset_files)):
    f = all_audioset_files[i]
    f = os.path.basename(f)[3:-4]
    h[f] = audios[i]

In [8]:
with open(os.path.join(audioset_data_path, "audioset_audios.pkl"), "wb") as f:
    pickle.dump(h, f)

In [11]:
# write examples DF
df = pd.DataFrame(all_examples)

In [13]:
df.tail()

,yt_id,start_time,end_time,tag_strings,laughter
19349,x11_NJ3hEVw,370.000,380.000,"[/m/0195fx, /m/01g50p, /m/06d_3, /m/07jdr, /m/...",False
19350,Wmjm2mu_7fY,240.000,250.000,"[/m/07yv9, /m/0h9mv, /m/0k4j]",False
19351,JC41M7RPSec,0.000,10.000,"[/m/04rlf, /m/06_y0by, /t/dd00129]",False
19352,eA2efaz3RSE,170.000,180.000,"[/m/015lz1, /m/04rlf, /m/07q4ntr, /t/dd00127]",False
19353,GpGKHdqpoLE,30.000,40.000,"[/m/01lsmm, /m/02g901, /m/04fgwm, /m/09x0r]",False


In [15]:
df.to_csv(os.path.join(audioset_data_path, 'audioset_examples.csv'), index=False)